In [1]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools

In [2]:
random.seed(1356) # set seed so that each play is the same

In [3]:
def hex_add(a, b):
    return Hex[a[0] + b[0], a[1] + b[1], a[2] + b[2]]

def hex_subtract(a, b):
    return Hex[a[0] - b[0], a[1] - b[1], a[2] - b[2]]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_directions[direction])

def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex[0]) + abs(hex[1]) + abs(hex[2])) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

In [4]:
hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

In [5]:
random.seed(1356)
side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 1 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4

In [6]:
def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

board = initializeBoard()
board, len(board)

({(-1, 0, 1): '',
  (-1, 1, 0): '',
  (0, -1, 1): '',
  (0, 0, 0): '',
  (0, 1, -1): '',
  (1, -1, 0): '',
  (1, 0, -1): ''},
 7)

In [7]:
def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

# wzobristboard

In [8]:
def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard


# bzobristboard

In [9]:
def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex


# zboard

In [10]:
def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [11]:
board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

boardToZobrist(board, zboard, wzobristboard, bzobristboard)

14024079292171647447

In [12]:
def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

In [13]:
# Form groups
# score = find_scores(board, 'w')
def findGroups(score, board):
    new_list = []
    group = {}
    group_state = {}
    group_count = 0

    for tile in score:
        counter = 0
        group[group_count] = 0
        if not tile in new_list:
            for direction in hex_directions:
                new_tile = tuple(map(sum, zip(tile, direction)))
                if new_tile in score:
                    new_list += [new_tile] # Already seen in group

                if score[tile] == board[direction]:
                    group[group_count] += counter
                    group_state.setdefault(tile, []).append(new_tile)

                counter += 1
        group_count += 1
    return group_state

In [14]:
# Check if lists can be grouped by at least 1 common member.
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [15]:
# Count total of each group
# group = findGroups(score, board)

def totalgroups(group):
    count_list = []

    for tile in group:
        for tile2 in group:
            if tile != tile2:
                intersect = list(Intersection(group[tile], group[tile2]))

                if len(intersect) != 0:
                    count_list += [len(list(set(group[tile] + group[tile2])))]
    return count_list


# print(totalgroups(group))

In [32]:
board = {
                 (-1, 0, 1): 'w',
                 (-1, 1, 0): 'b',
                 (0, -1, 1): 'b',
                 (0, 0, 0): '',
                 (0, 1, -1): 'b',
  
                 (1, -1, 0): 'b',
                 (1, 0, -1): 'w'}

fullBoard(board)

True

In [17]:
score = find_scores(board, 'b') #score of player1
score2 = find_scores(board, 'w') #score of player2

group = findGroups(score, board)
group2 = findGroups(score2, board)

count_list = totalgroups(group)
count_list2 = totalgroups(group2)

print(totalgroups(group), totalgroups(group2))

[] []


In [18]:
# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

result(count_list, count_list2)

(0, 0)

In [19]:
# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [20]:
# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [21]:
def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

In [22]:
def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

In [23]:
#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [24]:
k

NameError: name 'k' is not defined

In [25]:
#Creating entire the game tree for the play
from copy import deepcopy
import time

def buildGameTree(board, depth):
    depth += 1
    gametree = {}
    tempscore = 1
    
    zboard = initializeBoardZobrist() # initial board, without any pieces
    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    for i in range(len(all_cells)):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        
        count = 0
        while not valid:
            if count < 1:
                valid = True
            cell = all_cells[count]
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]
            
            #Make two valid moves
            if validMove(board2, x, y):
                if validMove(board2, x2, y2):
                    makeMove(board2, [x, y], 'b')

                    makeMove(board2, [x2, y2], 'w')
                    valid = True
            
                    # Create a zobrist key for the new board/childnode
                    zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

                    # If the board is full, tell the score
                    if fullBoard(board2):
                        
                        score = find_scores(board2, 'b') #score of player1
                        score2 = find_scores(board2, 'w') #score of player2

                        group = findGroups(score, board2)
                        group2 = findGroups(score2, board2)

                        count_list = totalgroups(group)
                        count_list2 = totalgroups(group2)
                        
                        player_one_total, player_two_total = result(count_list, count_list2)
                        
                        # Did player 1 win, lose or draw?
                        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]

                        #Store the final game state
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {game_state}])
                        
                    else:
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {}])#childTrees(board2, depth)]) # store full board + temporary score + horizon effect?
            
            count += 1
#             break
        all_cells = all_cells[count:]
    return gametree

start = time.time()
gametree = buildGameTree(board, 0)#13)
end = time.time()
print('Time: ', end - start)
gametree

Time:  0.004496097564697266


{14024079292171647447: [[765384773236229749,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): 'w',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [4743683394420103188,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): 'w',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [13524441802323967158,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): 'w',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [5985562699070635757,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): 'w',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [1850940307804494616,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): 'w',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [15004330490200760220,
   {(-1, 0, 1): 'b',
  

In [30]:
def childNodes(gametree): # pass full tree
    
    for key in gametree:
        for node in gametree[key]:
            newBoard = node[1]
            newDepth = node[2]

            newChild = buildGameTree(newBoard, newDepth) 
            node[4] = newChild
            childNodes(node[4])
                
    return gametree

start = time.time()
childNodes(gametree)
end = time.time()
print('Time: ', end - start)

{14024079292171647447: [[765384773236229749,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): 'w',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {765384773236229749: [[3585104155712041760,
      {(-1, 0, 1): 'b',
       (-1, 1, 0): 'w',
       (0, -1, 1): 'b',
       (0, 0, 0): 'w',
       (0, 1, -1): '',
       (1, -1, 0): '',
       (1, 0, -1): ''},
      2,
      1,
      {3585104155712041760: [[15667646708959168288,
         {(-1, 0, 1): 'b',
          (-1, 1, 0): 'w',
          (0, -1, 1): 'b',
          (0, 0, 0): 'w',
          (0, 1, -1): 'b',
          (1, -1, 0): 'w',
          (1, 0, -1): ''},
         3,
         1,
         {}],
        [1223680409285247908,
         {(-1, 0, 1): 'b',
          (-1, 1, 0): 'w',
          (0, -1, 1): 'b',
          (0, 0, 0): 'w',
          (0, 1, -1): 'b',
          (1, -1, 0): '',
          (1, 0, -1): 'w'},
         3,
         1,
         {}],
        [8906344357997116477,
       

In [ ]:
# Create path from root end node
def createPath(gametree, path):
    
    if type(gametree) != set:
        for node in gametree:
            tempPath = []
            rnd = random.randint(0, len(gametree[node])-1) # pick random node
            selected_node = gametree[node][rnd]

            path += [(selected_node[0], selected_node[2])] #add node to path
            createPath(selected_node[4], path) # check next node
    else:
        path += [gametree]
    
    return path

paths = []
for i in range(100):
    start = time.time()
    paths += [createPath(gametree, [])]
    end = time.time()
#     print('Time: ', end - start)
    
paths

In [ ]:
# def mcts(gametree, numsamples, wincountperinstance, tp_table, previous):
    
#     for i in range(numsamples):
        
#         if gametree != {}:
#             rnd_keys = random.choice(list(gametree))
#             nodes = gametree[rnd_keys]
#             rnd = random.randint(0, len(nodes) - 1)
#             instance = nodes[rnd]
            
#             if instance[4] != {}:
#                 if type(instance[4]) != set:
#                     instance[3] += 1
#                     mcts(instance[4], numsamples, wincountperinstance, tp_table, instance[0])
#                 else:
#                     gamestate = list(instance[4])
#                     if instance[4] == {'w'}:
#                         instance[3] = 999
#                         wincountperinstance.setdefault(instance[0], []).append(999)
#                     if instance[4] == {'l'}:
#                         instance[3] = -999
#                         wincountperinstance.setdefault(instance[0], []).append(-999)
#                     if instance[4] == {'d'}:
#                         instance[3] = -499
#                         wincountperinstance.setdefault(instance[0], []).append(-499)
            
# #             [depth, previous, next, score]
#             tp_table.setdefault(instance[0], []).append(instance[2])
#             tp_table.setdefault(instance[0], []).append(previous)
#             tp_table.setdefault(instance[0], []).append(instance[3])
            
# #             break

#     return gametree, wincountperinstance, tp_table

# # mcts_pl = mcts(gametree[14024079292171647447], 2)

# for i in range(100):
#     start = time.time()
#     mcts_pl = mcts(gametree, 100, {}, {}, list(gametree)[0])
#     end = time.time()
#     print('Time: ', end - start)
# # mcts_pl

In [ ]:
gametree[14024079292171647447][0]

In [ ]:
def updateTT(path, tt):
    
    gamescore = path[len(path)-1][1]
    
    for i in range(len(path)):
        
        node = path[i][0]
        tt[node][1] += path[i][1]
        
    return tt

In [ ]:
def countGameResults(gametree, depth, newdepth, oldkey, statecount, numsamples):
    
    if type(gametree) != list:
#         for key in gametree:
        for i in range(numsamples): # random pick a play
            rnd_key = random.choice(list(gametree))
            nodes = gametree[rnd_key]
            rnd = random.randint(0, len(nodes) - 1)
            node = nodes[rnd]
            
            if depth == newdepth:
                oldkey = rnd_key
            
            if type(node[4]) != set:
                countGameResults(node[4], depth, newdepth+1, oldkey, statecount, 2)
                depth += 1
            else:
                state = list(node[4])[0]
                if oldkey in statecount:
                    if state == 'w':
                        statecount[oldkey][0] += 1
                    elif state == 'l':
                        statecount[oldkey][1] += 1
                    else:
                        statecount[oldkey][2] += 1      
                else:
                    statecount[oldkey] = [1, 1, 1, node[2], node[3]]
                    
    return statecount

gamestate = countGameResults(gametree, 0, 0, 14024079292171647447, {14024079292171647447 : [1, 1, 1]}, 20000)
gamestate

In [ ]:
from anytree import Node, RenderTree, search

def createTree(gametree, root):
    
    if type(gametree) != set:
        for key in gametree:
            for node in gametree[key]:
                newnode = Node(node[0], parent=root)
                if type(node[4]) == set:
                    Node(node[2],  parent = newnode)
                    Node(node[3],  parent = newnode)
                    Node(node[4],  parent = newnode)
                createTree(node[4], newnode)
    return root


udo = createTree(gametree, Node(14024079292171647447))

for pre, fill, node in RenderTree(udo):
    print("%s%s        %s%s" % (pre, node.name, node.height, node.depth))

In [ ]:
udo.height, udo.depth #path to the leaf, depth)

In [ ]:
# Node('/14024079292171647447/765384773236229749/3585104155712041760').siblings
# udo.descendants
leaves = list(search.findall(udo, filter_=lambda node: node.name in ({'w'}, {'l'},{'d'})))
gamestates = []
for d in leaves:
    gamestates += [str(d).strip(')').split('/')[1:]]

gamestates

In [ ]:
countwins = {}

for game in gamestates:
    
    for i in range(len(game)-1):
        g = game[i]
        
        if g in countwins:
            if game[-1] == '{\'w\'}"':
                countwins[g][0] += 1
            elif game[-1] == '{\'l\'}"':
                countwins[g][1] += 1
            else:
                countwins[g][2] += 1 
        else:
            countwins[g] = [0, 0, 0]
#         countwins.setdefault(g, []).append(1)
        
countwins

In [ ]:
game[-1]

In [ ]:
def findTerminal(tempTree, keys, tt):
    path = []
    for key in tempTree: # For every parent
        for node in tempTree[key]: # For every child
            if type(node) == list:
                
                keys += [(key, node[2], node[3])]
                path += [(key, node[3])]
                tt[key] = [node[2], node[3]] #depth, score
                
                if type(node[4]) != set:
                    findTerminal(node[4], keys, tt) 
                else:
                    updateTT(path, tt)
        
    return tt, keys
        
tempTree = mcts_pl[0]
tt, keys = findTerminal(tempTree, [], {})
tt

In [ ]:
import numpy as np

# sort on score, highest first
a = np.array(mcts_pl[0][14024079292171647447])
a[a[:,3].argsort()[::-1]][:, 3]
#     )

# Checking Whether the Computer Can Win in One Move

In [ ]:
# Here is the algorithm for our Tic-Tac-Toe AI:
# First, check if we can win in the next move.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, computerLetter, i)
    if isWinner(boardCopy, computerLetter):
        return i

# Checking Whether the Player Can Win in One Move

In [ ]:
# Check if the player could win on their next move and block them.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, playerLetter, i)
    if isWinner(boardCopy, playerLetter):
        return i

# Checking Whether the Board Is Full

In [ ]:
def isBoardFull(board):
# Return True if every space on the board has been taken. Otherwise, return False.
for i in range(1, 10):
    if isSpaceFree(board, i):
        return False
    return True

# The Game Loop

In [ ]:
print('Welcome to Omega!')

In [ ]:
# Choosing the Player’s Mark and Who Goes First
playerLetter, computerLetter = inputPlayerLetter()
turn = whoGoesFirst()
print('The ' + turn + ' will go first.')
gameIsPlaying = True

In [ ]:
while gameIsPlaying:
    if turn == 'player':
        # Player's turn
        drawBoard(theBoard)
        move = getPlayerMove(theBoard)
        makeMove(theBoard, playerLetter, move)

    if isWinner(theBoard, playerLetter):
        drawBoard(theBoard)
        print('Hooray! You have won the game!')
        gameIsPlaying = False

    else:
        if isBoardFull(theBoard):
            drawBoard(theBoard)
            print('The game is a tie!')
            break

        else:
            turn = 'computer'

    else:
        # Computer's turn
        move = getComputerMove(theBoard, computerLetter)
        makeMove(theBoard, computerLetter, move)
        if isWinner(theBoard, computerLetter):
            drawBoard(theBoard)
            print('The computer has beaten you! You lose.')
            gameIsPlaying = False
        else:
            if isBoardFull(theBoard):
                drawBoard(theBoard)
                print('The game is a tie!')
                break
            else:
                turn = 'player'

# Asking the Player to Play Again

In [ ]:
print('Do you want to play again? (yes or no)')
if not input().lower().startswith('y'):
    break

In [ ]:
# def printBoard(board):
#     line = ''
#     space = 0
#     for y in range(0, N):
#         for x in range(-N, N*2):
#             try:
#                 line += Hex[(x, y, -x-y)]
#             except:
#                 line += '_'
        
#     return line
# printBoard(Hex)